In [2]:
import pandas as pd
import json
import pprint
import numpy as np
import os

import string
import unicodedata
import re

import matplotlib.pyplot as plt 

from nltk.sentiment.vader import SentimentIntensityAnalyzer
#analyser = SentimentIntensityAnalyzer()

import nltk
# from nltk.tokenize.t import ToktokTokenizer
# tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

from gensim import corpora, models
from keras import layers, models, optimizers
from sklearn.decomposition import LatentDirichletAllocation
from yellowbrick.classifier import ClassificationReport

pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

#preprocessing pipeline
#Pipeline models features like word count, tfidf, word density, word embeddings (GloVe)

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from keras.preprocessing import text, sequence
import xgboost




/Users/i504969/anaconda3/envs/py27/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
hs = pd.read_csv('hatespeech.csv', encoding="ISO-8859-1",index_col=6, keep_default_na=False)
#print(hs.head())

orig = pd.read_csv('NAACL_SRW_2016.csv', index_col=0, header=None)
orig.index.name = 'ID'
orig = orig.rename(columns={1: 'Class'})
orig.index = orig.index.astype(str)
#print(orig.head())

#merging the two dataframes
hs = pd.merge(hs, orig, how='inner', left_index=True, right_index=True)
#print(hs.head())
df = hs

In [4]:
df.head()

,Unnamed: 0,Authors,Descriptions,Favorite Count,Follower Count,Hashtags,Locations,Time Tweeted,Tweets,User Mentions,Class
ID,,,,,,,,,,,
319399851215433729,2004,CreatrixKali,Literary Creatrix for Alternative SpiritMag. Composer of articles of quirk and sometimes even remotely interesting status updates. And I'm in a book! Link below,0,169,"[{'text': 'MKR', 'indices': [37, 41]}]",Australia,2013-04-03 10:43:53,Oh yeah Colin! Smash those girls! :D #MKR,[],sexism
320817818222358529,5148,quincepoacher,"AKA Queenotisblue. Sociology, politics, policy, food, Gen Xer, Coburger & grumpy old lady. Social justice, education & employment stuff.",0,425,"[{'text': 'MKR', 'indices': [68, 72]}]",Melbourne,2013-04-07 08:38:23,It's insane they keep bringing people back. When will this show end #MKR,[],none
324114200450437120,5167,MarkTramby,"Mad keen Manly Sea Eagles supporter, love red wine (the better stuff), cricket ( all forms) and my wonderful family",0,330,"[{'text': 'MKR', 'indices': [16, 20]}, {'text': 'TheVoiceAu', 'indices': [92, 103]}]",Brisbane,2013-04-16 10:57:01,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu","[{'screen_name': 'berkeley_eagle', 'name': 'john G', 'id': 33840083, 'id_str': '33840083', 'indices': [0, 15]}]",none
326286656854454273,5221,BinnyD,"Reclusive nillionaire Whovian who enjoys television, arts, crafts and poking dead things with a stick.",0,211,"[{'text': 'MKR', 'indices': [18, 22]}]","NSW, Australia",2013-04-22 10:49:35,*sigh* oh Colin ? #MKR,[],none
381988216292655104,3612,YesYoureSexist,Inspired by @YesYoureRacist.,0,24041,[],,2013-09-23 03:47:42,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2","[{'screen_name': 'brian_day15', 'name': 'Brian Day', 'id': 392181216, 'id_str': '392181216', 'indices': [3, 15]}]",sexism


In [5]:
df_short = df[['Tweets','Class']]
# df = df.dropna(subset=['Label'])
df_short = df_short.dropna()
df_short.columns = ['data','label']

In [6]:
df_short.head()

,data,label
ID,,
319399851215433729,Oh yeah Colin! Smash those girls! :D #MKR,sexism
320817818222358529,It's insane they keep bringing people back. When will this show end #MKR,none
324114200450437120,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",none
326286656854454273,*sigh* oh Colin ? #MKR,none
381988216292655104,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",sexism


In [7]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "i'm": "i am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expand_contractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

def strip_mentions_hashtags(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

def high_precision(text):
    for token in text:
        words = text.split()
        assert words.isLower() == word.isUpper()
        
        

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    #text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


In [8]:
df = df_short

# data cleaning and processing
df['data'] = df['data'].apply(lambda x : strip_links(x))
df['data'] = df['data'].apply(lambda x : expand_contractions(x))
df['data'] = df['data'].apply(lambda x : strip_mentions_hashtags(x))
df['data'] = df['data'].apply(lambda x : remove_special_characters(x))

In [9]:
df

,data,label
ID,,
319399851215433729,oh yeah colin smash those girls d,sexism
320817818222358529,it is insane they keep bringing people back when will this show end,none
324114200450437120,eagle this shit show has more comebacks than johnny farnham ok back to the,none
326286656854454273,sigh oh colin,none
381988216292655104,rt day15 i swear i am not sexist but i honestly just cannot stand the woman college football announcer on espn2,sexism
381989167564996609,science is only good when it says men cannot be monogamous or some shit amirite,none
381989237072990208,rt call me sexist but i think some women are seriously lacking knowledge l when it comes to feminism,sexism
381989256404533251,rt call me sexist but females really need to stop acting like they are big football fans just stop,sexism
381989513691529216,rt the mine is no place for a woman to work call me sexist but a woman as a rock driller is not practical,sexism


In [10]:
df = df.drop_duplicates()
df = df.dropna()

In [11]:
#splitting dataset into train and validation set (80% - 20% default)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['data'], df['label'])


#Label Encoder converts yes and no into 1 and 0

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

#Count vectorizer will calculate count of every word in text data and will ignore number and whitespaces

count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])

#using the count vector defined above, we'll transform our existing text data into train_x_count where every row will indicate 
#tweet and every column will represent count of word indexed at that loc

train_x_count = count_vector.transform(train_x)
valid_x_count = count_vector.transform(valid_x)

#word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)


#ngram level tf-idf
tfidf_ngram = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', ngram_range=(2,3) ,max_features=5000)
tfidf_ngram.fit(df['data'])
xtrain_tfidf_ngram = tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram = tfidf_ngram.transform(valid_x)

from keras.preprocessing import text, sequence
token = text.Tokenizer()
token.fit_on_texts(df['data'])
word_index = token.word_index

train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=1000)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=1000)

In [11]:
from __future__ import division
import sys,json,math
import os
import numpy as np

import numpy as np


def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

def load_word2vec(filename):
    # Returns a dict containing a {word: numpy array for a dense word vector} mapping.
    # It loads everything into memory.
    
    w2vec={}
    with open(filename,"r") as f_in:
        for line in f_in:
            line_split=line.replace("\n","").split()
            w=line_split[0]
            vec=np.array([float(x) for x in line_split[1:]])
            w2vec[w]=vec
    return w2vec

In [12]:
word_to_vec_dict = loadGloveModel('glove.twitter.27B.200d.txt')

import numpy
emb_matrix = numpy.zeros((len(word_index)+1, 200))
for word, i in word_index.items():
    emb_vect = word_to_vec_dict.get(word)
    if emb_vect is not None:
        emb_matrix[i] = emb_vect

Loading Glove Model
('Done.', 1193514, ' words loaded!')


In [13]:
dataset_train = pd.DataFrame(train_x)
dataset_valid = pd.DataFrame(valid_x)


In [14]:
dataset_for_topic_model = pd.concat([dataset_train, dataset_valid], axis=0)


In [15]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset_for_topic_model['data'])
lda = LatentDirichletAllocation(random_state=0)
doc_topic_matrix = lda.fit_transform(X)


In [16]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()


In [17]:
dataset_train['comp'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['compound'])
dataset_train['pos'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['pos'])
dataset_train['neu'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['neu'])
dataset_train['neg'] = dataset_train['data'].apply(lambda x : analyser.polarity_scores(x)['neg'])

dataset_valid['comp'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['compound'])
dataset_valid['pos'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['pos'])
dataset_valid['neu'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['neu'])
dataset_valid['neg'] = dataset_valid['data'].apply(lambda x : analyser.polarity_scores(x)['neg'])

In [18]:
# combine sentiment features with count vectorizer features
train_x_count = pd.DataFrame(np.concatenate((train_x_count.toarray(), dataset_train[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[:len(train_x)]), axis=1))
valid_x_count = pd.DataFrame(np.concatenate((valid_x_count.toarray(), dataset_valid[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[len(train_x):]), axis=1))

# combine sentiment features with tf-idf features
xtrain_tfidf = pd.DataFrame(np.concatenate((xtrain_tfidf.toarray(), dataset_train[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[:len(train_x)]), axis=1))
xvalid_tfidf = pd.DataFrame(np.concatenate((xvalid_tfidf.toarray(), dataset_valid[['comp', 'pos', 'neu', 'neg']].values, doc_topic_matrix[len(train_x):]), axis=1))

In [19]:
from sklearn.metrics import f1_score

def train_model(classifier, feature_vector_train,label, feature_vector_valid, is_neural_net = False):
    classifier.fit(feature_vector_train, label)
    prediction = classifier.predict(feature_vector_valid)
    from sklearn.metrics import confusion_matrix
    if not is_neural_net:
        print(confusion_matrix(valid_y, prediction))
    if is_neural_net:
        prediction = prediction.argmax(axis=-1)
    return metrics.accuracy_score(prediction, valid_y), f1_score(prediction, valid_y,average=None)


In [20]:
count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['data'])
nb_train_x_count = count_vector.transform(train_x)
nb_valid_x_count = count_vector.transform(valid_x)
tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['data'])
nb_xtrain_tfidf = tfidf_vect.transform(train_x)
nb_xvalid_tfidf = tfidf_vect.transform(valid_x)
n


clf = naive_bayes.MultinomialNB()
accuracy, f1 = train_model(clf, nb_train_x_count, train_y, nb_valid_x_count)
print("Naive Bayes (Count Vectors)",accuracy, f1)
accuracy, f1 = train_model(clf, nb_xtrain_tfidf, train_y, nb_xvalid_tfidf)
print("Naive Bayes (TDIDF)", accuracy, f1)

[[2307  169  153]
 [ 135  346    3]
 [ 345    4  450]]
('Naive Bayes (Count Vectors)', 0.793200408997955, array([0.85192024, 0.68993021, 0.6405694 ]))
[[2568   36   25]
 [ 294  190    0]
 [ 551    0  248]]
('Naive Bayes (TDIDF)', 0.7684049079754601, array([0.85004965, 0.53521127, 0.46268657]))


In [33]:
clf = linear_model.LogisticRegression(verbose=1)
accuracy, f1 = train_model(clf, train_x_count, train_y, valid_x_count)
print("LR using CV", accuracy, f1)
accuracy, f1 = train_model(clf, xtrain_tfidf, train_y, xvalid_tfidf)
print("LR using CV", accuracy, f1)

[LibLinear][[2434   86  113]
 [ 149  334    4]
 [ 318    2  472]]
('LR using CV', 0.8282208588957055, array([0.87965305, 0.73487349, 0.68356264]))
[LibLinear][[2495   68   70]
 [ 207  275    5]
 [ 417    0  375]]
('LR using CV', 0.8039366053169734, array([0.86752434, 0.6626506 , 0.60386473]))


In [ ]:
accuracy, f1 = train_model(xgboost.XGBClassifier(verbose=1), train_x_count, train_y, valid_x_count)
print("xgboost using CV", accuracy, f1)
accuracy, f1 = train_model(xgboost.XGBClassifier(verbose=1), xtrain_tfidf, train_y, xvalid_tfidf)
print("xgboost using TFIDF", accuracy, f1)

In [39]:
#Shallow Neural Networks

def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier

In [40]:
classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy, f1 = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/20
11736/11736 [==============================] - 5s 419us/step - loss: 0.4249
Epoch 2/20
11736/11736 [==============================] - 4s 361us/step - loss: -0.4221
Epoch 3/20
11736/11736 [==============================] - 4s 378us/step - loss: -0.7532
Epoch 4/20
11736/11736 [==============================] - 4s 343us/step - loss: -0.9797
Epoch 5/20
11736/11736 [==============================] - 4s 339us/step - loss: -1.1788
Epoch 6/20
11736/11736 [==============================] - 4s 336us/step - loss: -1.3251
Epoch 7/20
11736/11736 [==============================] - 4s 338us/step - loss: -1.4424
Epoch 8/20
11736/11736 [==============================] - 4s 339us/step - loss: -1.5396
Epoch 9/20
11736/11736 [==============================] - 4s 338us/step - loss: -1.6192
Epoch 10/20
11736/11736 [==============================] - 4s 337us/step - loss: -1.6858
Epoch 11/20
11736/11736 [==============================] - 4s 337us/step - loss: -1.7446
Epoch 12/20
11736/11736 [======

In [44]:
classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy, f1 = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/2
11736/11736 [==============================] - 5s 415us/step - loss: 0.4216
Epoch 2/2
11736/11736 [==============================] - 4s 339us/step - loss: -0.4022
('NN, Ngram Level TF IDF Vectors', 0.6730572597137015)


In [46]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 200, weights=[emb_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print(accuracy)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 1/2
11736/11736 [==============================] - 56s 5ms/step - loss: 0.0461
Epoch 2/2
11736/11736 [==============================] - 54s 5ms/step - loss: -0.3812
(0.6730572597137015, array([0.80458365, 0.        , 0.        ]))


In [13]:
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical


model = Sequential()

#Add layers to model
model.add(Dense(100, activation='relu', input_shape = (70,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

early_stop = EarlyStopping(patience=2) 
#train_seq_x, train_y, valid_seq_x
#fit model
model.fit(train_seq_x, train_y, validation_split=0.3, epochs=20, callbacks=[early_stop])

ValueError: Error when checking input: expected dense_11_input to have shape (1000,) but got array with shape (70,)

In [ ]:
max_words = 1000
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

y_train = utils.to_categorical(train_y, 3)
y_test = utils.to_categorical(valid_y, 3)


model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(train_seq_x, y_train,
                    batch_size=32,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 10562 samples, validate on 1174 samples
Epoch 1/3


In [ ]:
new_class = []

for i in df.Class:
    if i == 'sexism':
        new_class.append(0)
    elif i == 'racism':
        new_class.append(1)
    else:
        new_class.append(2)

#print(list(df.Class)[0:10])
#print(new_class[0:10])

target=to_categorical(new_class)
n_cols = predictors.shape[1]